In [1]:
import pandas as pd
import os

In [12]:
def clean_data(input_file, output_file):
#lee los archivos de la ruta que le especificamos
    try:
        df=pd.read_csv(input_file)
    except FileNotFoundError: 
        print(FileNotFoundError)
        print("El archivo especificado no se ha encontrado, indique una ruta válida")
        return

    #Eliminar las filas que tengan valor NaN
    clean_df = df.dropna()

    #Escribe un los datos filtrados en un archivo nuevo 
    clean_df.to_csv(output_file, index=False)
    print("Datos limpios guardados como ", output_file)

In [13]:
def main(): 
    while True: 
        # Solicitar al usuario que ingrese el nombre del archivo de entrada
        input_file = input("Ingrese la ruta y nombre del archivo CSV de entrada: ")
    
         # Solicitar al usuario que ingrese el nombre del archivo de salida
        output_file = input("Ingrese la ruta y el nombre del archivo CSV de salida para los datos limpios: ")
    
        if input_file == output_file: 
            print("Por favor, indique nombres para los archivos que sean distintivos")
        else: 
            # Llamar a la función para limpiar los datos
            clean_data(input_file, output_file)
            break

In [15]:
if __name__ == "__main__":
    main()

Ingrese la ruta y nombre del archivo CSV de entrada:  datosCSVleer/default_exp_Session1_Shimmer_F693_Calibrated_SD.csv
Ingrese la ruta y el nombre del archivo CSV de salida para los datos limpios:  datosCSVleer/default_exp_Session1_Shimmer_F693_Calibrated_SD_f.csv


Datos limpios guardados como  datosCSVleer/default_exp_Session1_Shimmer_F693_Calibrated_SD_f.csv
